In [1]:
# load U-Net model and model weights 

from simple_unet_model import simple_unet_model 
from keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from skimage import measure, color, io
import pandas as pd

IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 1

def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = get_model()
model.load_weights('lamellipodia.hdf5')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 256, 256, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
# Conduct U-Net semantic segmentation

import imageio.v2 as imageio

def main():
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Segmentation/T30_Mock_0018_segmented/"
    os.makedirs(outPath)
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/T30_Mock_0018/"

    # iterate through the images of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, image_path)
        image_to_segment = cv2.imread(input_path, 0)
        
        # process the image
        image_to_segment = Image.fromarray(image_to_segment)
        image_to_segment = image_to_segment.resize((IMG_HEIGHT, IMG_WIDTH))
        test_img_norm = np.expand_dims(normalize(np.array(image_to_segment), axis=1),2)
        test_img_norm = test_img_norm[:,:,0][:,:,None]
        test_img_input = np.expand_dims(test_img_norm, 0)

        # segment 
        segmented = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)

        # save to folder 
        fullpath = os.path.join(outPath, 'segmented_'+image_path)
        imageio.imwrite(fullpath, segmented)

if __name__ == '__main__':
    main()

In [10]:
# watershed segmentation 

import sys
import cv2
from matplotlib import pyplot as plt
import pyclesperanto_prototype as cle
from skimage import io
import pandas as pd

def main():
    
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/Watershed/"
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Segmentation/T30_Mock_0018_segmented/"
    
    data = []
    image = []

    # iterate through the images of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, image_path)
        image_to_segment = cv2.imread(input_path)
        
        # process image
        img_grey = image_to_segment[:,:,0]
        
        # otsu thresholding
        ret1, thresh = cv2.threshold(img_grey, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        # remove noise
        kernel = np.ones((3,3),np.uint8)
        opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

        # watershed
        sure_bg = cv2.dilate(opening,kernel,iterations=10)
        dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
        ret2, sure_fg = cv2.threshold(dist_transform, 0.2*dist_transform.max(),255,0)
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg,sure_fg)
        ret3, markers = cv2.connectedComponents(sure_fg)
        markers = markers+10
        markers[unknown==255] = 0
        markers = cv2.watershed(image_to_segment, markers)
        
        # properties
        props = measure.regionprops_table(markers, intensity_image=img_grey, 
                              properties=['area',
                                          'mean_intensity'])
        df = pd.DataFrame(props)
        count = (len(df))

        # save count measurements
        image.append(image_path)
        data.append(count)

    # export measurements
    df = pd.DataFrame(list(zip(image , data)), columns =['Image', 'Number of lamellipodia'])
    basename_without_ext = os.path.basename(os.path.dirname(path)) 
    df.to_excel(outPath + basename_without_ext + "_Watershed.xlsx", index=False)
    
if __name__ == '__main__':
    main()

In [11]:
# calculate average values

import pandas as pd

def main():
    
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/Watershed/"
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/Watershed/"
    
    folder = []
    x = []
    s = []
    
    # iterate through excel files of the folder
    for file in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, file)
        df = pd.read_excel(input_path)
        
        # calculate mean
        mean = df["Number of lamellipodia"].mean()
        se = df["Number of lamellipodia"].sem()
        
        # save values
        folder.append(file)
        x.append(mean)
        s.append(se)
    
    # export values
    df = pd.DataFrame(list(zip(folder, x, s)), columns = ['File', 'Average number of lamellipodia', 'Standard error'])
    df.to_excel(outPath + "Watershed_Compiled_Results.xlsx", index=False)
                    
if __name__ == '__main__':
    main()